In [1]:
# creating dataframe 
import matplotlib.pyplot as plt 
import pandas as pd
from sqlalchemy import create_engine

database_name = 'tcom'
table_name= 'xdr_data'

connection_params = { "host": "localhost", "user": "postgres", "password": "1234",
                    "port": "5432", "database": database_name}

engine = create_engine(f"postgresql+psycopg2://{connection_params['user']}:{connection_params['password']}@{connection_params['host']}:{connection_params['port']}/{connection_params['database']}")

# str or SQLAlchemy Selectable (select or text object)
sql_query = 'SELECT * FROM xdr_data'

df = pd.read_sql(sql_query, con= engine)